<a href="https://colab.research.google.com/github/BAfsharmanesh/CS8945-project/blob/main/manifold_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -q install gdown
!gdown --folder "https://drive.google.com/drive/folders/1IUaQJgQY1i9TZDWnY9x_eE8HBBSu22T_?usp=drive_link" -O /content/dataset

Retrieving folder contents
Retrieving folder 1WCUV6adZ8K-OQrb0qYLrirlJoypAzfaj rep-llama2-7b
Processing file 1cQj67jvxtmX8MMOe75EVlNfZ3gNHTcVw rep-model.layers.0.post_attention_layernorm.npy
Processing file 1V7XV118zYQebVDiI3Rx-AlyLduBK4v94 rep-model.layers.1.post_attention_layernorm.npy
Processing file 1wDUZA_XrmIH8xWGsZ4683e6wC_kks5Zi rep-model.layers.2.post_attention_layernorm.npy
Processing file 1bUz1wBErSSEmt3Lstitx6Bftpa_rRMxM rep-model.layers.3.post_attention_layernorm.npy
Processing file 1A7lSiuYKEPvM_2O_KDonObE9fXxbJIcY rep-model.layers.4.post_attention_layernorm.npy
Processing file 1r8o4udUnjDhmCwHHWAm6ueC4i-n2EoUz rep-model.layers.5.post_attention_layernorm.npy
Processing file 1agkLK4egDgpQ4S3rJUcDb6M8F0Fld_XY rep-model.layers.6.post_attention_layernorm.npy
Processing file 1i0MsXSwmam8MtX1AHgIB5ozrDqtGcQNk rep-model.layers.7.post_attention_layernorm.npy
Processing file 1AeGUUyutVLhWzKJjBQOvlMo1fM4t1Lvy rep-model.layers.8.post_attention_layernorm.npy
Processing file 1B4NxZ4xv

In [3]:
import os
import numpy as np

# Directory containing the .npy files
folder_path = '/content/dataset/rep-llama2-7b'

# List all files in the directory
all_files = os.listdir(folder_path)

# Filter for .npy files
npy_files = [f for f in all_files if f.endswith('.npy')]

# Dictionary to store the loaded data
loaded_data = {}

# Load each .npy file
for file_name in npy_files:
    file_path = os.path.join(folder_path, file_name)
    if 'layers' in file_name:
      layer_number = file_name.split('.')[2]
      loaded_data[layer_number] = np.load(file_path)
    else:
      loaded_data['norm'] = np.load(file_path)

# You can access the loaded data using the file names as keys
# For example, to access the data from 'rep-model.norm.npy':
# print(loaded_data['rep-model.norm.npy'].shape)

print(f"Loaded {len(loaded_data)} .npy files.")

Loaded 33 .npy files.


In [26]:
print(loaded_data["0"].shape)
print(loaded_data["31"].shape)


(11855, 4096)
(11855, 4096)


Perform dimensionality reduction using PCA and Isomap on each layer of the loaded data, visualize the 2D results for each layer and technique, and then interpret and compare the PCA and Isomap visualizations across all layers, focusing on geometric analysis.

## Process Each Layer with PCA and Isomap


In [27]:
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap
print("Imported PCA and Isomap.")

Imported PCA and Isomap.


In [30]:
pca_isomap_results = {}

# Get the keys from loaded_data, convert to integers, sort, and convert back to strings
layer_keys_sorted = sorted([int(k) for k in loaded_data.keys() if k != 'norm'])

for layer_key_int in layer_keys_sorted:
    layer_key = str(layer_key_int)
    data = loaded_data[layer_key]

    print(f"\nProcessing Layer {layer_key} with shape {data.shape}")

    # 1. Apply PCA directly to reduce to 2 dimensions
    pca_direct = PCA(n_components=2)
    pca_result_direct = pca_direct.fit_transform(data)

    # 2. Apply PCA to reduce dimensionality while retaining 95% variance as preprocessing for Isomap
    pca_95_variance = PCA(n_components=0.95)
    pca_preprocessed_data = pca_95_variance.fit_transform(data)
    print(f"  PCA (95% variance) reduced data to shape: {pca_preprocessed_data.shape[1]} dimensions.")

    # 3. Apply Isomap to the PCA-preprocessed data
    # Calculate n_neighbors, ensuring it's at least 2 and less than n_samples
    n_samples = pca_preprocessed_data.shape[0]
    n_neighbors = min(10, n_samples - 1)
    if n_neighbors < 2 and n_samples > 1:
        n_neighbors = 2
    elif n_samples == 1:
        print("  Cannot apply Isomap to a single sample.")
        isomap_result = np.array([[]]) # Store an empty array or handle as appropriate
    else:
        isomap = Isomap(n_components=2, n_neighbors=n_neighbors)
        isomap_result = isomap.fit_transform(pca_preprocessed_data)

    # Store results
    pca_isomap_results[layer_key] = {
        'pca_direct': pca_result_direct,
        'isomap_after_pca': isomap_result
    }

    print(f"  Shape after direct PCA: {pca_result_direct.shape}")
    print(f"  Shape after Isomap (following 95% PCA): {isomap_result.shape}")

print(f"\nDimensionality reduction completed for {len(pca_isomap_results)} layers.")


Processing Layer 0 with shape (11855, 4096)
  PCA (95% variance) reduced data to shape: 204 dimensions.


/usr/local/lib/python3.12/dist-packages/sklearn/manifold/_isomap.py:384: UserWarning: The number of connected components of the neighbors graph is 7 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
/usr/local/lib/python3.12/dist-packages/scipy/sparse/_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])
/usr/local/lib/python3.12/dist-packages/scipy/sparse/_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])
/usr/local/lib/python3.12/dist-packages/scipy/sparse/_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])
/usr/local/lib/python3.12/dist-pa

  Shape after direct PCA: (11855, 2)
  Shape after Isomap (following 95% PCA): (11855, 2)

Processing Layer 1 with shape (11855, 4096)
  PCA (95% variance) reduced data to shape: 465 dimensions.


/usr/local/lib/python3.12/dist-packages/sklearn/manifold/_isomap.py:384: UserWarning: The number of connected components of the neighbors graph is 4 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
/usr/local/lib/python3.12/dist-packages/scipy/sparse/_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])
/usr/local/lib/python3.12/dist-packages/scipy/sparse/_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])
/usr/local/lib/python3.12/dist-packages/scipy/sparse/_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])
/usr/local/lib/python3.12/dist-pa

  Shape after direct PCA: (11855, 2)
  Shape after Isomap (following 95% PCA): (11855, 2)

Processing Layer 2 with shape (11855, 4096)
  PCA (95% variance) reduced data to shape: 650 dimensions.


/usr/local/lib/python3.12/dist-packages/sklearn/manifold/_isomap.py:384: UserWarning: The number of connected components of the neighbors graph is 3 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
/usr/local/lib/python3.12/dist-packages/scipy/sparse/_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])
/usr/local/lib/python3.12/dist-packages/scipy/sparse/_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])
/usr/local/lib/python3.12/dist-packages/scipy/sparse/_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])


  Shape after direct PCA: (11855, 2)
  Shape after Isomap (following 95% PCA): (11855, 2)

Processing Layer 3 with shape (11855, 4096)
  PCA (95% variance) reduced data to shape: 960 dimensions.


/usr/local/lib/python3.12/dist-packages/sklearn/manifold/_isomap.py:384: UserWarning: The number of connected components of the neighbors graph is 2 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
/usr/local/lib/python3.12/dist-packages/scipy/sparse/_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])


  Shape after direct PCA: (11855, 2)
  Shape after Isomap (following 95% PCA): (11855, 2)

Processing Layer 4 with shape (11855, 4096)
  PCA (95% variance) reduced data to shape: 1215 dimensions.
  Shape after direct PCA: (11855, 2)
  Shape after Isomap (following 95% PCA): (11855, 2)

Processing Layer 5 with shape (11855, 4096)
  PCA (95% variance) reduced data to shape: 1452 dimensions.
  Shape after direct PCA: (11855, 2)
  Shape after Isomap (following 95% PCA): (11855, 2)

Processing Layer 6 with shape (11855, 4096)
  PCA (95% variance) reduced data to shape: 1695 dimensions.
  Shape after direct PCA: (11855, 2)
  Shape after Isomap (following 95% PCA): (11855, 2)

Processing Layer 7 with shape (11855, 4096)
  PCA (95% variance) reduced data to shape: 1803 dimensions.
  Shape after direct PCA: (11855, 2)
  Shape after Isomap (following 95% PCA): (11855, 2)

Processing Layer 8 with shape (11855, 4096)
  PCA (95% variance) reduced data to shape: 1824 dimensions.
  Shape after direct

Visualize the 2D PCA and Isomap results for each of the 32 layers. For each layer, generate two scatter plots: one for the direct PCA output and one for the Isomap output (applied after PCA preprocessing).

In [31]:
import os
import matplotlib.pyplot as plt

# Create a directory to save the plots
output_dir = 'manifold_visualizations'
os.makedirs(output_dir, exist_ok=True)

for layer_key, results in pca_isomap_results.items():
    pca_data = results['pca_direct']
    isomap_data = results['isomap_after_pca']

    # Create a figure with two subplots: one for PCA and one for Isomap
    fig, axes = plt.subplots(1, 2, figsize=(16, 6)) # Adjust figsize for better display

    # Plot Direct PCA results on the first subplot
    axes[0].scatter(pca_data[:, 0], pca_data[:, 1], alpha=0.5)
    axes[0].set_title(f'Layer {layer_key} - Direct PCA Visualization')
    axes[0].set_xlabel('Principal Component 1')
    axes[0].set_ylabel('Principal Component 2')
    axes[0].grid(True)

    # Plot Isomap results (after PCA preprocessing) on the second subplot
    # Ensure isomap_data is not empty before plotting
    if isomap_data.shape[0] > 0:
        axes[1].scatter(isomap_data[:, 0], isomap_data[:, 1], alpha=0.5)
        axes[1].set_title(f'Layer {layer_key} - Isomap (PCA-preprocessed) Visualization')
        axes[1].set_xlabel('Isomap Component 1')
        axes[1].set_ylabel('Isomap Component 2')
        axes[1].grid(True)
    else:
        axes[1].set_title(f'Layer {layer_key} - Isomap (PCA-preprocessed) Visualization')
        axes[1].text(0.5, 0.5, 'Isomap not applicable or empty result', horizontalalignment='center', verticalalignment='center', transform=axes[1].transAxes)

    plt.tight_layout() # Adjust subplot parameters for a tight layout

    # Save the figure instead of displaying it inline immediately
    filename = os.path.join(output_dir, f'layer_{layer_key}_pca_isomap.png')
    plt.savefig(filename)
    plt.close(fig) # Close the figure to free up memory

print(f"All {len(pca_isomap_results)} layers' PCA and Isomap visualizations saved to the '{output_dir}' directory.")

All 32 layers' PCA and Isomap visualizations saved to the 'manifold_visualizations' directory.
